# Collect Spotify Data

## Load the libraries

In [1]:
import pandas as pd
import spotipy
import spotipy.oauth2 as oauth2
import csv

# Load the API token
credentials = oauth2.SpotifyClientCredentials(
        client_id='bb2c8bd8bf164c6da9c2e474047b941b',
        client_secret="06ed1e8f3fda43e3a829a830f03ce777")

token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)

print(spotify)

## Input Singer Name as search key word

In [2]:
#Input the Singer Name from twitter csv
info =pd.read_csv('twitter_singer.csv')
info

,Unnamed: 0,singer_ID,singer_name,singer_url,singer_description,singer_followers_count
0,0,73992972,Avril Lavigne,https://t.co/niyvkKkHrn,BMG Records. Freesolo / Vector Management. PMK...,21722762
1,1,34507480,Ariana Grande,https://t.co/76qmPxOH2C,"thank u, next",61210749
2,2,21447363,KATY PERRY,https://t.co/VxeqPgXTF4,Love. Light.,107002594
3,3,79293791,Rihanna,https://t.co/6HarIyKc3f,Throwing all the shade! @fentybeauty,89893415
4,4,119509520,Chris Brown,https://t.co/jCtn90KqAF,#Undecided available everywhere now! SOMEWHERE...,29841144
5,5,27260086,Justin Bieber,https://t.co/r6Zj8zy1lK,Let's make the world better.,105116612
6,6,14230524,Lady Gaga,https://t.co/6CDfvJPhW0,NaN,78135950
7,7,24886570,Maroon 5,http://t.co/3UhWOxHCRb,This is the Official Twitter page for Maroon 5...,14697963
8,8,100220864,Bruno Mars,https://t.co/lrnSFFfTtz,NaN,42492751
9,9,17919972,Taylor Swift,https://t.co/RSxHoIri2k,The reputation Stadium Tour is streaming now o...,83270778


In [3]:
singers = info['singer_name']
print (singers)

0    Avril Lavigne
1    Ariana Grande
2       KATY PERRY
3          Rihanna
4      Chris Brown
5    Justin Bieber
6        Lady Gaga
7         Maroon 5
8       Bruno Mars
9     Taylor Swift
Name: singer_name, dtype: object


## Using Singer Name to seach the related song

In [4]:
# Create a new csv to store the data related to song
with open('spotify_song.csv', 'w', newline='') as csvfile:
    fieldnames = ['singer_name', 
                  'singer_spotify_id',
                  'song_name',
                  'song_id',
                  'duration_ms',
                  'song_popularity',
                  'album_name',
                  'album_id'
                  ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
# put the data in the csv
    # Make a loop to search all the singer name in the dataset  
    for singer in singers:
        
        # The result of the data releated to singer name          
        results = spotify.search(q=singer, type='track')
        result = results['tracks']
        
        # The most related song 
        track = result['items'][0]
        
        # The artist information of the song
        artist = track['artists'][0] 
        creator_id = artist['id']
        creator_name = artist['name']
        
        # Create a loop in case the singer of the song is the second singer       
        if creator_name == singer:
            artist = track['artists'][0] 
            creator_id = artist['id']
            creator_name = artist['name']
       
        elif singer == 'Justin Bieber':
                track = result['items'][3]
                artist = track['artists'][0] 
                creator_id = artist['id']
                creator_name = artist['name'] 
        else:
            track = result['items'][1]
            artist = track['artists'][0] 
            creator_id = artist['id']
            creator_name = artist['name']
            
            if creator_name != singer:
                track = result['items'][2]
                artist = track['artists'][0] 
                creator_id = artist['id']
                creator_name = artist['name']
            
        
            
        # The information of the song         
        song_id = track['id']
        song_name = track['name']
        duration_ms = track['duration_ms']
        song_popularity = track['popularity']
        
        # The album information of the song
        album = track['album']
        album_name = album['name']
        album_id = album['id']
        
        # Put all the data we get into the csv
        writer.writerow({'singer_name': creator_name, 
                         'singer_spotify_id':creator_id,
                         'song_name':song_name,
                         'song_id':song_id,
                         'duration_ms':duration_ms,
                         'song_popularity':song_popularity,
                         'album_name':album_name,
                         'album_id':album_id,
                         })
        print(singer,":", creator_name,':',album_id)

Avril Lavigne : Avril Lavigne : 3zXjR3y2dUWklKmmp6lEhy
Ariana Grande : Ariana Grande : 2fYhqwDWXjbpjaIJPEfKFw
KATY PERRY : Katy Perry : 5MQBzs5YlZlE28mD9yUItn
Rihanna : Rihanna : 3Q149ZH46Z0f3oDR7vlDYV
Chris Brown : Chris Brown : 0JOT3Ff4gJ939hqdeEpsZv
Justin Bieber : Justin Bieber : 6Fr2rQkZ383FcMqFyT7yPr
Lady Gaga : Lady Gaga : 4sLtOBOzn4s3GDUv3c5oJD
Maroon 5 : Maroon 5 : 75iQSBSaztFIAun9qLLCnb
Bruno Mars : Bruno Mars : 4PgleR09JVnm3zY1fW3XBA
Taylor Swift : Taylor Swift : 6DEjYFkNZh67HP7R9PSZvv


## Using Album ID to seach the related album

In [5]:
# Get the album id in song csv
album_info =pd.read_csv('spotify_song.csv')
album_ids = album_info['album_id']
# singerNameS = album_info['singer_name']

In [6]:
# Create a new csv to store the data related to album
with open('spotify_album.csv', 'w', newline='') as csvfile:
    fieldnames = ['singer_name',
                  'singer_spotify_id',
                  'album_name',
                  'album_id',
                  'album_date',
                  'total_tracks']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
# put the data in the csv
    # Make a loop to search all the album id in the dataset    
    for album_idd in album_ids:        
        # The result of the data releated to album
        album_item = spotify.album(album_idd)
        
        # The artist information of the album
        bb = album_item['artists'][0]
        album_creator = bb['name']
        album_creator_id = bb['id']
        
        # The album information of the song
        album_name = album_item['name']
        album_id = album_item['id']
        album_date = album_item['release_date']
        total_tracks = album_item['total_tracks']
        
        
        # Put all the data we get into the csv
        writer.writerow({
                         'singer_name': album_creator,
                         'singer_spotify_id': album_creator_id,
                         'album_name': album_name,
                         'album_id': album_id,
                         'album_date': album_date, 
                         'total_tracks': total_tracks
                        })
        print(album_idd,":", album_id,":", album_creator)

3zXjR3y2dUWklKmmp6lEhy : 3zXjR3y2dUWklKmmp6lEhy : Avril Lavigne
2fYhqwDWXjbpjaIJPEfKFw : 2fYhqwDWXjbpjaIJPEfKFw : Ariana Grande
5MQBzs5YlZlE28mD9yUItn : 5MQBzs5YlZlE28mD9yUItn : Katy Perry
3Q149ZH46Z0f3oDR7vlDYV : 3Q149ZH46Z0f3oDR7vlDYV : Rihanna
0JOT3Ff4gJ939hqdeEpsZv : 0JOT3Ff4gJ939hqdeEpsZv : Chris Brown
6Fr2rQkZ383FcMqFyT7yPr : 6Fr2rQkZ383FcMqFyT7yPr : Justin Bieber
4sLtOBOzn4s3GDUv3c5oJD : 4sLtOBOzn4s3GDUv3c5oJD : Lady Gaga
75iQSBSaztFIAun9qLLCnb : 75iQSBSaztFIAun9qLLCnb : Maroon 5
4PgleR09JVnm3zY1fW3XBA : 4PgleR09JVnm3zY1fW3XBA : Bruno Mars
6DEjYFkNZh67HP7R9PSZvv : 6DEjYFkNZh67HP7R9PSZvv : Taylor Swift


## Using Creator ID to seach the related singer

In [7]:
# Get the singer id in album csv
artist_info =pd.read_csv('spotify_album.csv')
creator_ids = artist_info['singer_spotify_id']

In [8]:
# Create a new csv to store the data related to album
with open('spotify_artist.csv', 'w', newline='') as csvfile:
    fieldnames = [
                  'singer_name', 
                  'singer_spotify_id',
                  'singer_spotify_followers',
                  'singer_spotify_popularity',
                  'total_albums',
                  'top_1_related_artist',
                  'top_2_related_artist',
                  'top_3_related_artist'
                 ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
# put the data in the csv
    # Make a loop to search all the singer id in the dataset
    for creator_id in creator_ids:
        
        # The result of the data releated to artist
        artist = spotify.artist(creator_id)
        
        # The artist information of the album
        
        artist_name = artist['name']
        
        creator_name == singer
        artist_id = artist['id']
        gg = artist['followers']
        artist_followers = gg['total']
        artist_popularity = artist['popularity']
        
        # The total albums of the singer has 
        uri = 'spotify:artist:'+ creator_id
        total_albums_results = spotify.artist_albums(uri, album_type='album')
        total_albums = total_albums_results['total']
        
        # The related artists of the singer
        related_artists = spotify.artist_related_artists(artist_id)
        #top 1 related Artists
        top1 = related_artists['artists'][0]
        top1_name= top1['name']
        #top 2 related Artists
        top2 = related_artists['artists'][1]
        top2_name= top2['name']
        #top 3 related Artists
        top3 = related_artists['artists'][2]
        top3_name= top3['name']
        
        # Put all the data we get into the csv
        writer.writerow({
                         'singer_name': artist_name, 
                         'singer_spotify_id': artist_id,
                         'singer_spotify_followers': artist_followers,
                         'singer_spotify_popularity': artist_popularity,                                           
                    
                         'total_albums': total_albums,
                         
                         'top_1_related_artist': top1_name,
                         'top_2_related_artist': top2_name,
                         'top_3_related_artist': top3_name,                
                        })
        print(creator_id,":", artist_id,':', artist_name)

0p4nmQO2msCgU4IF37Wi3j : 0p4nmQO2msCgU4IF37Wi3j : Avril Lavigne
66CXWjxzNUsdJxJ2JdwvnR : 66CXWjxzNUsdJxJ2JdwvnR : Ariana Grande
6jJ0s89eD6GaHleKKya26X : 6jJ0s89eD6GaHleKKya26X : Katy Perry
5pKCCKE2ajJHZ9KAiaK11H : 5pKCCKE2ajJHZ9KAiaK11H : Rihanna
7bXgB6jMjp9ATFy66eO08Z : 7bXgB6jMjp9ATFy66eO08Z : Chris Brown
1uNFoZAHBGtllmzznpCI3s : 1uNFoZAHBGtllmzznpCI3s : Justin Bieber
1HY2Jd0NmPuamShAr6KMms : 1HY2Jd0NmPuamShAr6KMms : Lady Gaga
04gDigrS5kc9YWfZHwBETP : 04gDigrS5kc9YWfZHwBETP : Maroon 5
0du5cEVh5yTK9QJze8zA0C : 0du5cEVh5yTK9QJze8zA0C : Bruno Mars
06HL4z0CvFAxyc27GXpf02 : 06HL4z0CvFAxyc27GXpf02 : Taylor Swift


## Related Singer's data

In [9]:
# Get the singer id in album csv
realted_artist_info =pd.read_csv('spotify_artist.csv')
top_1_related = realted_artist_info['top_1_related_artist']

In [10]:
# Create a new csv to store the data related to song
with open('spotify_song.csv', 'a+', newline='') as csvfile:
    fieldnames = [
                  'singer_name', 
                  'singer_spotify_id',
                  'song_name',
                  'song_id',
                  'duration_ms',
                  'song_popularity',
                  'album_name',
                  'album_id'
                  ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
#     writer.writeheader()
#     writer = csv.writer(csvfile)
    
# put the data in the csv
    # Make a loop to search all the singer name in the dataset  
    for singer in top_1_related:
        
        # The result of the data releated to singer name          
        results = spotify.search(q=singer, type='track')
        result = results['tracks']
        
        # The most related song 
        track = result['items'][0]
        
        # The information of the song         
        song_id = track['id']
        song_name = track['name']
        duration_ms = track['duration_ms']
        song_popularity = track['popularity']
        
        # The artist information of the song
        artist = track['artists'][0] 
        creator_id = artist['id']
        creator_name = artist['name']
        
        # Create a loop in case the singer of the song is the second singer       
        if creator_name == singer:
            artist = track['artists'][0] 
            creator_id = artist['id']
            creator_name = artist['name']
        
        
        elif len(track['artists']) < 2:
            creator_name = 'Brain'
            
        else:
            artist = track['artists'][1] 
            creator_id = artist['id']
            creator_name = artist['name']
        
            
        # The album information of the song
        album = track['album']
        album_name = album['name']
        album_id = album['id']
        
        # Put all the data we get into the csv
        writer.writerow({'singer_name': creator_name, 
                         'singer_spotify_id':creator_id,
                         'song_name':song_name,
                         'song_id':song_id,
                         'duration_ms':duration_ms,
                         'song_popularity':song_popularity,
                         'album_name':album_name,
                         'album_id':album_id,
                         })
        print(singer,":", creator_name)

The Veronicas : The Veronicas
Camila Cabello : Camila Cabello
Avril Lavigne : Avril Lavigne
Nicki Minaj : Nicki Minaj
Jeremih : Jeremih
Niall Horan : Niall Horan
Carly Rae Jepsen : Carly Rae Jepsen
OneRepublic : OneRepublic
Jason Derulo : Jason Derulo
Selena Gomez : Selena Gomez


In [11]:
# Get the singer id in album csv
realted_artist_info =pd.read_csv('spotify_artist.csv')
top_2_related = realted_artist_info['top_2_related_artist']

In [12]:
# Create a new csv to store the data related to song
with open('spotify_song.csv', 'a+', newline='') as csvfile:
    fieldnames = ['singer_name', 
                  'singer_spotify_id',
                  'song_name',
                  'song_id',
                  'duration_ms',
                  'song_popularity',
                  'album_name',
                  'album_id'
                  ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
# put the data in the csv
    # Make a loop to search all the singer name in the dataset  
    for singer in top_2_related:
        
        # The result of the data releated to singer name          
        results = spotify.search(q=singer, type='track')
        result = results['tracks']
        
        # The most related song 
        track = result['items'][0]
        
        # The information of the song         
        song_id = track['id']
        song_name = track['name']
        duration_ms = track['duration_ms']
        song_popularity = track['popularity']
        
        # The artist information of the song
        artist = track['artists'][0] 
        creator_id = artist['id']
        creator_name = artist['name']
        
        # Create a loop in case the singer of the song is the second singer       
        if creator_name == singer:
            artist = track['artists'][0] 
            creator_id = artist['id']
            creator_name = artist['name']
        
        
        elif len(track['artists']) < 2:
            creator_name = 'Brain'
            
        else:
            artist = track['artists'][1] 
            creator_id = artist['id']
            creator_name = artist['name']
        
            
        # The album information of the song
        album = track['album']
        album_name = album['name']
        album_id = album['id']
        
        # Put all the data we get into the csv
        writer.writerow({'singer_name': creator_name, 
                         'singer_spotify_id':creator_id,
                         'song_name':song_name,
                         'song_id':song_id,
                         'duration_ms':duration_ms,
                         'song_popularity':song_popularity,
                         'album_name':album_name,
                         'album_id':album_id,
                         })
        print(singer,":", creator_name)

Ashlee Simpson : Ashlee Simpson
ZAYN : ZAYN
Kelly Clarkson : Kelly Clarkson
Beyoncé : Beyoncé
Trey Songz : Trey Songz
ZAYN : ZAYN
Troye Sivan : Troye Sivan
The Script : The Script
Jessie J : Jessie J
Demi Lovato : Demi Lovato


In [13]:
# Get the singer id in album csv
realted_artist_info =pd.read_csv('spotify_artist.csv')
top_3_related = realted_artist_info['top_3_related_artist']

In [14]:
# Create a new csv to store the data related to song
with open('spotify_song.csv', 'a+', newline='') as csvfile:
    fieldnames = ['singer_name', 
                  'singer_spotify_id',
                  'song_name',
                  'song_id',
                  'duration_ms',
                  'song_popularity',
                  'album_name',
                  'album_id'
                  ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
       
# put the data in the csv
    # Make a loop to search all the singer name in the dataset  
    for singer in top_3_related:
        
        # The result of the data releated to singer name          
        results = spotify.search(q=singer, type='track')
        result = results['tracks']
        
        # The most related song 
        track = result['items'][0]
        
        # The information of the song         
        song_id = track['id']
        song_name = track['name']
        duration_ms = track['duration_ms']
        song_popularity = track['popularity']
        
        # The artist information of the song
        artist = track['artists'][0] 
        creator_id = artist['id']
        creator_name = artist['name']
        
        # Create a loop in case the singer of the song is the second singer       
        if creator_name == singer:
            artist = track['artists'][0] 
            creator_id = artist['id']
            creator_name = artist['name']
        
        
        elif len(track['artists']) < 2:
            creator_name = 'Cash Cash'
            
        elif singer == 'Juicy J' :
            creator_name = 'Juicy J'
            
        else:
            artist = track['artists'][1] 
            creator_id = artist['id']
            creator_name = artist['name']
        
            
        # The album information of the song
        album = track['album']
        album_name = album['name']
        album_id = album['id']
        
        # Put all the data we get into the csv
        writer.writerow({'singer_name': creator_name, 
                         'singer_spotify_id':creator_id,
                         'song_name':song_name,
                         'song_id':song_id,
                         'duration_ms':duration_ms,
                         'song_popularity':song_popularity,
                         'album_name':album_name,
                         'album_id':album_id,
                         })
        print(singer,":", creator_name)

Hilary Duff : Hilary Duff
Dua Lipa : Dua Lipa
Selena Gomez & The Scene : Selena Gomez & The Scene
Miley Cyrus : Miley Cyrus
Mario : Mario
Liam Payne : Liam Payne
Kim Petras : Kim Petras
Train : Train
Sean Kingston : Sean Kingston
Miley Cyrus : Miley Cyrus


In [15]:
# Get the album id in song csv
album_info =pd.read_csv('spotify_song.csv')
album_ids = album_info['album_id']

In [16]:
# Create a new csv to store the data related to album
with open('spotify_album.csv', 'w', newline='') as csvfile:
    fieldnames = [
        'singer_name',
                  'singer_spotify_id',
                  'album_name',
                  'album_id',
                  'album_date',
                  'total_tracks'
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    

# put the data in the csv
    # Make a loop to search all the album id in the dataset
    for album_idd in album_ids:
        
        # The result of the data releated to album
        album_item = spotify.album(album_idd)
        
        # The artist information of the album
        bb = album_item['artists'][0]
        album_creator = bb['name']
        album_creator_id = bb['id']
        
        # The album information of the song
        album_name = album_item['name']
        album_id = album_item['id']
        album_date = album_item['release_date']
        total_tracks = album_item['total_tracks']
        
        
        # Put all the data we get into the csv
        writer.writerow({
                         'singer_name': album_creator,
                         'singer_spotify_id': album_creator_id,
                         'album_name': album_name,
                         'album_id': album_id,
                         'album_date': album_date, 
                         'total_tracks': total_tracks
                        })
        print(album_idd,":", album_id)

3zXjR3y2dUWklKmmp6lEhy : 3zXjR3y2dUWklKmmp6lEhy
2fYhqwDWXjbpjaIJPEfKFw : 2fYhqwDWXjbpjaIJPEfKFw
5MQBzs5YlZlE28mD9yUItn : 5MQBzs5YlZlE28mD9yUItn
3Q149ZH46Z0f3oDR7vlDYV : 3Q149ZH46Z0f3oDR7vlDYV
0JOT3Ff4gJ939hqdeEpsZv : 0JOT3Ff4gJ939hqdeEpsZv
6Fr2rQkZ383FcMqFyT7yPr : 6Fr2rQkZ383FcMqFyT7yPr
4sLtOBOzn4s3GDUv3c5oJD : 4sLtOBOzn4s3GDUv3c5oJD
75iQSBSaztFIAun9qLLCnb : 75iQSBSaztFIAun9qLLCnb
4PgleR09JVnm3zY1fW3XBA : 4PgleR09JVnm3zY1fW3XBA
6DEjYFkNZh67HP7R9PSZvv : 6DEjYFkNZh67HP7R9PSZvv
6aL2SwYj5kSEvIcYORHP37 : 6aL2SwYj5kSEvIcYORHP37
6hCR3zQtRH0IgookOYt771 : 6hCR3zQtRH0IgookOYt771
3zXjR3y2dUWklKmmp6lEhy : 3zXjR3y2dUWklKmmp6lEhy
03KcW1ZhaSnj8pIk1LUNQs : 03KcW1ZhaSnj8pIk1LUNQs
7DMyQuDPe8xzjC0UDSDa96 : 7DMyQuDPe8xzjC0UDSDa96
1xJ7jIK1tT0aVoJw1fPE6r : 1xJ7jIK1tT0aVoJw1fPE6r
29blfJv8AddJrjuG3DpE13 : 29blfJv8AddJrjuG3DpE13
49vus1VCWc7AbgDmKYb8SC : 49vus1VCWc7AbgDmKYb8SC
2e5CxfyEwBW115beiwh7Mc : 2e5CxfyEwBW115beiwh7Mc
0ZHyvPke0EcVt6rYA8Ins3 : 0ZHyvPke0EcVt6rYA8Ins3
0w11HUk0hE4WZ95Bvp6vNI : 0w11HUk0hE4WZ95